In [9]:
import pandas as pd

Import the snoDB file

In [10]:
snodb = pd.read_csv('dataset/snoDB_All_V2.0.csv')
snodb.head()

,Unnamed: 0,snodb_id,ensembl_id,refseq_id,hgnc_id,ncbi_id,snornabase_id,snorna_atlas_id,snopy_id,rna_central_id,...,snrna_targets,lncrna_targets,protein_coding_targets,snorna_targets,mirna_targets,trna_targets,ncrna_targets,pseudogene_targets,other_targets,is_expressed
0,0,snoDB0001,ENSG00000221083,NaN,NaN,NaN,NaN,NaN,Homo_sapiens300648,URS00006F7F32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,1,snoDB0002,ENSG00000252404,NaN,NaN,NaN,NaN,NaN,NaN,URS000066189C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,2,snoDB0003,ENSG00000239149,NR_003025.1,HGNC:32653,677885.0,SR0000241,snoID_0545,NaN,URS0000056589,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
3,3,snoDB0004,ENSG00000252969,NaN,NaN,NaN,NaN,NaN,NaN,URS00006DED88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,4,snoDB0005,ENSG00000251866,NR_135613.1,HGNC:52237,107397391.0,NaN,snoID_0577,NaN,URS00006C4777;URS0000BC45E9,...,U12.1-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [11]:
snodb.columns

Index(['Unnamed: 0', 'snodb_id', 'ensembl_id', 'refseq_id', 'hgnc_id',
       'ncbi_id', 'snornabase_id', 'snorna_atlas_id', 'snopy_id',
       'rna_central_id', 'rfam_id', 'snodb_legacy_id', 'host_gene_id', 'chr',
       'start', 'end', 'strand', 'gene_name', 'synonyms', 'box_type',
       'conservation_snorna_atlas', 'conservation_phastcons', 'length',
       'sequence', 'host_chr', 'host_start', 'host_end', 'host_strand',
       'host_gene_name', 'host_synonyms', 'host_biotype', 'host_function',
       'target_count', 'target_biotypes', 'rrna_targets', 'snrna_targets',
       'lncrna_targets', 'protein_coding_targets', 'snorna_targets',
       'mirna_targets', 'trna_targets', 'ncrna_targets', 'pseudogene_targets',
       'other_targets', 'is_expressed'],
      dtype='object')

Import the complete graph of the network

In [12]:
net_graph = pd.read_csv('../../Tools_Expand_Genes_Network/networkOutput/95/Complete_Graph.csv')
net_graph.drop_duplicates(inplace=True)
net_graph.head()

,NodeA,NodeB,frel
0,@AP1G1,@RPL10,0.963609
1,@AP1G1,@DDX39B,0.990533
2,@CFDP1,@NCL,0.980600
4,@CWF19L1,@RPL21P28,0.957732
5,@CWF19L1,@AP1G1,0.953662


Clean the names of the nodes

In [13]:
net_graph.NodeA = net_graph.NodeA.apply(lambda x: x.replace('@', ''))
net_graph.NodeB = net_graph.NodeB.apply(lambda x: x.replace('@', ''))
net_graph.head()

,NodeA,NodeB,frel
0,AP1G1,RPL10,0.963609
1,AP1G1,DDX39B,0.990533
2,CFDP1,NCL,0.980600
4,CWF19L1,RPL21P28,0.957732
5,CWF19L1,AP1G1,0.953662


Extract all available nodes in the dataset

In [14]:
nodes = set()

for index, row in net_graph.iterrows():
    nodes.add(row.NodeA)
    nodes.add(row.NodeB)

Extract the targets for the snoRNAs 

In [15]:
snodb = snodb[snodb['gene_name'].isin(nodes)]
snodb = snodb[['ensembl_id', 'gene_name', 'snorna_targets']]
snodb_no_interactor = snodb[snodb['snorna_targets'].isnull()]
snodb_interactor = snodb[snodb['snorna_targets'].notnull()]

In [16]:
len(set(snodb_no_interactor.gene_name.unique()) - set(snodb_interactor.gene_name.unique()))

59

In [17]:
" & ".join(set(snodb_no_interactor.gene_name.unique()) - set(snodb_interactor.gene_name.unique()))

'SNORD45A & SNORA24 & SNORD97 & SNORA12 & SNORD14C & SNORD14A & SNORD34 & SNORA71C & SNORA63 & SNORD18A & SNORD55 & SNORD36A & SNORD4B & SNORD99 & SNORD36C & SNORD87 & SNORA61 & SNORD36B & SNORD58C & SNORA57 & SNORA64 & SNORA32 & SNORD32A & SNORD49B & SNORD69 & SNORD29 & SNORD110 & SNORD17 & SNORD33 & SNORD35B & SNORA38 & SNORD8 & SNORD37 & SNORD102 & SNORA52 & SNORD5 & SNORD86 & SNORA53 & SNORA8 & SNORD26 & SNORD111B & SNORD71 & SNORD44 & SNORD54 & SNORA1 & SNORA49 & SNORD74 & SNORD42B & SNORD14D & SNORA23 & SNORD96A & SNORD68 & SNORD57 & SNORA16A & SNORD58A & SNORD58B & SNORA48 & SNORD1C & SNORD65'

Create the dictionary for the targets

In [19]:
more_than_one = dict()
for sno, count in snodb_interactor.gene_name.value_counts().items():
    if count > 1:
        more_than_one[sno] = []

sno_dict = dict()

for sno in more_than_one:
    sno_dict[sno] = []

for index, row in snodb_interactor.iterrows():
    if row.gene_name in more_than_one:
        more_than_one[row.gene_name].append(row.snorna_targets)
    sno_dict[row.gene_name] = row.snorna_targets

for sno in more_than_one:
    sno_dict[sno] = ';'.join(more_than_one[sno])

for key in sno_dict:
    sno_dict[key] = sno_dict[key].split(';')

sno_dict

{'SNORA75': ['SNORD13',
  'SNORD20',
  'SNORD22',
  'SNORD27',
  'SNORD30',
  'SNORD31',
  'SNORD46',
  'SNORD75',
  'SNORD76',
  'SNORD62A'],
 'SNORA70': ['SNORA65',
  'SNORA72',
  'SNORD27',
  'SNORD30',
  'SNORD43',
  'SNORD50A',
  'SNORD52',
  'SNORD80',
  'SNORD88C'],
 'SNORA14B': ['SNORD103A', 'SNORD62A'],
 'SNORD38A': ['U17_SNRNA'],
 'SNORA73A': ['RNU105C', 'SNORA73B', 'SNORA74A', 'SNORD25', 'SNORD7'],
 'SNORA73B': ['RNU105C', 'SNORA73A'],
 'SNORA44': ['SNORA22', 'SNORA34'],
 'SNORD53': ['SCARNA12',
  'SNORD104',
  'SNORD117',
  'SNORD118',
  'SNORD13',
  'SNORD20',
  'SNORD22',
  'SNORD31',
  'SNORD41',
  'SNORD47',
  'SNORD48',
  'SNORD59A',
  'SNORD60',
  'SNORD74',
  'SNORD94'],
 'SNORD89': ['SNORD46'],
 'SNORD94': ['SCARNA10',
  'SCARNA12',
  'SNORD117',
  'SNORD12C',
  'SNORD20',
  'SNORD22',
  'SNORD27',
  'SNORD31',
  'SNORD41',
  'SNORD53',
  'SNORD81'],
 'SNORD20': ['SCARNA10',
  'SCARNA12',
  'SNORA75',
  'SNORD117',
  'SNORD13',
  'SNORD22',
  'SNORD27',
  'SNORD30',

In [20]:
our_snos = {"SNORA1","SNORA12","SNORA14B","SNORA16A","SNORA21","SNORA23","SNORA24","SNORA32","SNORA38","SNORA44","SNORA48","SNORA49","SNORA52","SNORA53","SNORA57","SNORA61","SNORA63","SNORA64","SNORA65","SNORA70","SNORA71C","SNORA73A","SNORA73B","SNORA75","SNORA78","SNORA8","SNORD10","SNORD102","SNORD104","SNORD105","SNORD105B","SNORD108","SNORD110","SNORD111B","SNORD113-3","SNORD113-4","SNORD114-1","SNORD114-13","SNORD114-14","SNORD114-19","SNORD114-20","SNORD114-21","SNORD115-23","SNORD115-32","SNORD116-13","SNORD119","SNORD12","SNORD12B","SNORD13","SNORD14A","SNORD14C","SNORD14D","SNORD15B","SNORD16","SNORD17","SNORD18A","SNORD1B","SNORD20","SNORD22","SNORD26","SNORD28","SNORD29","SNORD32A","SNORD33","SNORD34","SNORD35A","SNORD36A","SNORD36B","SNORD38A","SNORD3A","SNORD3D","SNORD41","SNORD42A","SNORD42B","SNORD44","SNORD45A","SNORD47","SNORD49B","SNORD4B","SNORD5","SNORD50A","SNORD52","SNORD53","SNORD54","SNORD55","SNORD56","SNORD57","SNORD58A","SNORD58C","SNORD59A","SNORD60","SNORD63","SNORD64","SNORD65","SNORD68","SNORD69","SNORD71","SNORD74","SNORD76","SNORD8","SNORD81","SNORD82","SNORD84","SNORD86","SNORD87","SNORD89","SNORD94","SNORD96A","SNORD97","SNORD99","SNORD1C","SNORD35B","SNORD37"}

not_expanded = {'PCAT4', 'SNORD114-14', 'RPL23', 'SNORD114-1', 'RPS3', 'SNORD114-19', 'RACK1', 'SNORD113-4', 'NAN', 'SNORD114-13', 'RPS11', 'RPS2', 'SNORD114-20', 'SNHG20', 'SNORD115-23', 'SNORD116-13', 'CDKN2B-AS1', 'SNORD113-3', 'SNORD115-32', 'SLC25A3', 'ZFAS1', 'HIF1A-AS2', 'SNORD114-21'}

our_snos = our_snos - not_expanded
new_sno_dict = dict()
count = 0
for sno in our_snos:
    if sno in sno_dict:
        count += 1
        new_sno_dict[sno] = sno_dict[sno]

count

43

Check if the connections are present in our graph

In [21]:
snodb_connections = dict()

for sno in new_sno_dict:
    for target in new_sno_dict[sno]:
        snodb_connections[(sno, target)] = 0
        for index, edge in net_graph.iterrows():
            if (sno == edge.NodeA and target == edge.NodeB) or (sno == edge.NodeA and target == edge.NodeB):
                snodb_connections[(sno, target)] += 1

snodb_connections

{('SNORD76', 'SNORA75'): 0,
 ('SNORD76', 'SNORD66'): 0,
 ('SNORD76', 'SNORD75'): 0,
 ('SNORD84', 'SNORD117'): 0,
 ('SNORA65', 'SNORA11'): 0,
 ('SNORA65', 'SNORA47'): 0,
 ('SNORA65', 'SNORA66'): 0,
 ('SNORA65', 'SNORA70'): 0,
 ('SNORA65', 'SNORD116-9'): 0,
 ('SNORA65', 'SNORD46'): 0,
 ('SNORA65', 'SNORD47'): 0,
 ('SNORA65', 'SNORD50A'): 0,
 ('SNORA65', 'SNORD56'): 0,
 ('SNORA65', 'SNORD80'): 0,
 ('SNORA65', 'SNORD83B'): 0,
 ('SNORD56', 'SNORA65'): 0,
 ('SNORD56', 'SNORD50A'): 0,
 ('SNORD60', 'SNORD49A'): 0,
 ('SNORD16', 'SNORA25'): 0,
 ('SNORA73B', 'RNU105C'): 0,
 ('SNORA73B', 'SNORA73A'): 1,
 ('SNORD104', 'SNORD103A'): 0,
 ('SNORD104', 'SNORD53'): 0,
 ('SNORD104', 'SNORD74'): 0,
 ('SNORD82', 'SNORD116-9'): 0,
 ('SNORD82', 'SNORD117'): 0,
 ('SNORD82', 'SNORD59A'): 0,
 ('SNORD82', 'SNORD81'): 0,
 ('SNORA14B', 'SNORD103A'): 0,
 ('SNORA14B', 'SNORD62A'): 0,
 ('SNORD119', 'SNORD115'): 0,
 ('SNORD3D', 'U3'): 0,
 ('SNORA75', 'SNORD13'): 0,
 ('SNORA75', 'SNORD20'): 0,
 ('SNORA75', 'SNORD22'): 

Print the connections that are present

In [22]:
for conn in snodb_connections:
    if snodb_connections[conn] > 0:
        print(conn)

('SNORA73B', 'SNORA73A')
('SNORD108', 'SNORD64')
('SNORD47', 'SNORD59A')
('SNORD47', 'SNORD60')
('SNORD20', 'SNORA75')
('SNORD12', 'SNORD12B')
('SNORD94', 'SNORD22')


In [23]:
count = 0

for key in snodb_connections:
    if snodb_connections[key] > 0:
        count += 1

count

7

In [24]:
core_graph = pd.read_csv('../../Tools_Expand_Genes_Network/networkOutput/95/Core_Graph.csv')
core_graph.NodeA = core_graph.NodeA.apply(lambda x: x.replace('@', ''))
core_graph.NodeB = core_graph.NodeB.apply(lambda x: x.replace('@', ''))
core_graph.drop_duplicates(inplace=True)
core_graph.head()

,NodeA,NodeB,frel
0,SNORD37,EEF2,0.9857


In [25]:
to_test = ["SNHG5", "SNORD50B", "SNHG5", "SNORD22", "SNHG1", "HSPA8", "SNORD14C", "EIF4A2", "SNORA63", "SNORA4", "RPS8", "SNORD46", "SNORD38A"]
count_present = dict()
for gene in to_test:
    for index, row in core_graph.iterrows():
        if gene == row.NodeA or gene == row.NodeB:
            if gene in count_present:
                count_present[gene].append((row.NodeA, row.NodeB))
            else:
                count_present[gene] = [(row.NodeA, row.NodeB)]

count_present

{}

In [26]:
to_test = ["SNHG20", "MEG8", "SNORD114-3", "SNORD3B-1", "SNORD3B-2", "SNORD3D", "SNORD3A", "SNORD3C", "MEG8", "SNORD114-9", "SNORD114-10", "SNHG20", "SNORD104", "SNORA50C"]
count_present = dict()
for gene in to_test:
    for index, row in core_graph.iterrows():
        if gene == row.NodeA or gene == row.NodeB:
            if gene in count_present:
                count_present[gene].append((row.NodeA, row.NodeB))
            else:
                count_present[gene] = [(row.NodeA, row.NodeB)]

for key in count_present:
    print(key, len(count_present[key]))

In [27]:
snodb = pd.read_csv('dataset/snoDB_All_V2.0.csv')
snodb = snodb[snodb.host_gene_name == 'TOMM20']
snodb.gene_name

30    SNORA14B
Name: gene_name, dtype: object